In [21]:
import pandas as pd 
import os
from google.cloud import bigquery
sentiment_dict = {0:'negative',1:'neutral',2:'positive'}
pd.set_option('max_colwidth', 800)
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "credentials_gpe-analytics.json"

In [5]:
client = bigquery.Client()

# Perform a query.
query = (
    'SELECT * FROM `gpe-analytics.project_google_arxr_analytics_20240109.view_dashboard_final_update_v2_distinct`'
)
query_job = client.query(query)  # API request
preds = query_job.to_dataframe()

In [53]:
client = bigquery.Client()

# Perform a query.
query = (
    'SELECT text,uid FROM `gpe-analytics.project_google_arxr_analytics_20240109.ar_xr_predictions_update`'
)
query_job = client.query(query)  # API request
preds_other = query_job.to_dataframe()



In [57]:
len(preds)-61894

121796

In [55]:
len(set(preds['text']) - set(preds_other['text']))

61894

In [ ]:
# Perform a query.
query = (
    'SELECT * FROM `gpe-analytics.project_google_arxr_analytics_20240109.view_dashboard_final_update_v2_distinct`'
)
query_job = client.query(query)  # API request
preds = query_job.to_dataframe()

In [ ]:
android_sentiment
neutral     0.783069
positive    0.175044
negative    0.035714
Name: count, dtype: float64
        
        
llm_label
positive    0.588624
neutral     0.405203
negative    0.006173
Name: count, dtype: float64

# Comparison of Android NLP and LLM Label for 

Google was showing 45.3% negative sentiment and Vuzix was showing 63% positive. 





Google Glasses Sentiment Discrepancies
@Tiffany when I look in the dashboard I see that google glasses has 41.4% negative from the llm_label. @Michelle doing a comparison between the llm_label and our out of box sentiment labels it is pretty comparable, with android_sentiment (our own NLP model) predicting relatively high negative numbers(~38%). Looking at the llm_labels when predicting sentiment, and when the llm_label disagrees with the out of box model, I would argue that the LLM label seems to be more accurate in terms of judging the text based on the entity specific sentiment, not just sentiment as a whole, which was the goal of this project. For reasons why these numbers seem high compared to previous sentiment numbers:

1. Our out of box sentiment models are not entity specific. To demonstrate how our out of box sentiment models work let's take two text examples:

"I love apple. Apple is great and amazing and wonderful. Android is the worst brand to ever grace the face of the earth." 

Out-of-box android sentiment model (non-entity specific): Labels this as positive. This is because there are more positive words than negative words in the text.

LLM model when asked to label the sentiment of the text based on the entitiy Android: Labels this as negative. 

"Can Microsoft succeed where Google failed? The company has recently filed a patent for augmented reality (AR)"

Out-of-box android sentiment model (non-entity specific): Labels this as neutral, this is because there are more neutral words/tones in the text than positive or negative.

LLM model when asked to label the sentiment of the text based on the entitiy Google Glasses: Labels this as negative, because contextually it is more negative towards Google. 


Therefore the extremes of positive and negatives may appear higher than normal compared to non-entity specific models, and even neutrals may appear to be lower than normal as it is now labeling the text based on the entity in question. 


2. The glasses query happened to capture a lot more negative language.

3. We are adding in news sources too, of which many of them did seem relatively negative to google glasses. Typically we don't include news as a source in the analyst dash

4. These numbers are correct and on social most people talk negatively about google glasses 

Moving forward, if the data does not seem truly negative (keeping in mind that that there is about 10-25% of the time both our own out of box models and the llm get the sentiment wrong), then we would need to adjust in the prompt to specific cases for why the text should not be labeled negative. In the image I shared comparing the android_sentiment model and the llm_label, this is a random sample of 15 out of 4073 negative observations. Of this sample, I would agree with the llm_label at least 80% of the time, disagreeing with the non-entity specific label 20% of the time or less.

Vuzix Glasses Sentiment Discrepancies
Looking at the dashboard I get 58.9% for the llm_label and 18% positive for android sentiment. The percentages are in less agreement with each other. Exploring the data to compare android sentiment and llm sentiment, it is a bit difficult to say which is right due the type of posts being pulled in. For vuzix, we only pulled in 2268 mentions. The majority of those mentions come from twitter (~70%) and of those twitter mentions most appear to be retweeets and promotional. Note that most promotional tweets e.g. "RT Vuzix and Join Forces to Redefine Smart with Printing. Read more about our latest news on integrating 3D printing into the process for Vuzix’s smart glasses via here: $vuzi	" tend to be more positive leaning, especially from an entity perspective. Moving forward, one possible options is to remove retweets (but then the sample size will get smaller), or consider all promotional tweets as neutral, which then the majority will be neutral because the majority of the posts are promotional. I showed the comparison of the android sentiment and llm label below, again note that the LLM model is labeling based on the sentiment specific to Vuzix glasses. The android sentiment model labels these all as neutrals because the majority appear to be promotional, where as the llm is labeling these as positive in respect to Vuzix. @Natalie do you have a particular way you would like to move forward with this example? 



If none of this made sense, please feel free to ask away with questions, and if anything else comes up in the data ensure you flag me so I can investigate. Thanks! 


In [49]:
vuzix_glasses['updated_source'].value_counts()/len(vuzix_glasses)

updated_source
twitter      0.701058
news         0.254850
reddit       0.033069
instagram    0.011023
Name: count, dtype: float64

In [50]:
vuzix_glasses.loc[vuzix_glasses['disagreement']==True].sample(n=10,random_state=42)[['text','updated_source','android_sentiment','llm_label']]

,text,updated_source,android_sentiment,llm_label
1850,RT Partner Spotlight: 's PxLens is a wearable headset powered by Vuzix smart glasses that enables surgeons to share a first-person perspective of medical procedures through real-time telepresence. Discover more: $vuzi,twitter,neutral,positive
793,"RT ""The acquisition of Moviynt further positions Vuzix as a software solutions provider capable of expanding access and interaction between wearable and handheld devices…"" PRESS RELEASE: |",twitter,neutral,positive
363,"Augmented Reality, Virtual Reality, Mixed Reality explainedNo, I mean an AR headset. Wait, make that an MR headset with a dial to switch between AR and VR at will… Despite years of headsets from big-name corporations like Google, Microsoft, Sony, Facebook/Meta, and more, most people still don't unde",news,negative,neutral
863,RT Join Vuzix at the Summit in just two weeks to immerse yourself in an innovative augmented reality remote support demo with Vuzix M400 smart glasses. Find us at Booth # 20! $vuzi,twitter,neutral,positive
1514,RT . customer success story with Vuzix and $vuzi,twitter,neutral,positive
586,excellent work! congrat and thanks... I will have a vuzix blade 2 device tomorrow. Do you think that would be compatible with it? Can I run it inside the smart glass or should I use it on Android phone and send the screen to the glass?,reddit,positive,neutral
1770,"RT CEO Paul Travers and Alex Westner, CEO of Vuzix partner Xander, join ‘This Week in Hearing’ via to discuss the innovative paired with Vuzix supporting people who are hard of hearing with real-time captioning. $VUZI",twitter,neutral,positive
1241,"RT CEO Spotlight ""Through expanding relationships with distributors such as Xpertinc, we continue to see numerous companies across a growing number of industry verticals turning to and embracing Vuzix solutions,"" Paul Travers. More here: $vuzi",twitter,neutral,positive
1853,Revolutionise emergency care with Vuzix Smart Glasses & Remote Expert.Learn more: info@net4.io or 0208 1940822. �� #EmergencyServices,twitter,neutral,positive
1301,RT Vuzix and Join Forces to Redefine Smart with Printing. Read more about our latest news on integrating 3D printing into the process for Vuzix’s smart glasses via here: $vuzi,twitter,neutral,positive


In [45]:
vuzix_glasses_positive.loc[vuzix_glasses_positive['disagreement']==True].sample(n=10,random_state=42)[['text','updated_source','android_sentiment','llm_label']]

,text,updated_source,android_sentiment,llm_label
1219,RT Vuzix AR Smart Glasses ‘Transformational’ for Surgeries ���� A world-renowned professor explains his surgical work with 's and efficacy in remote instruction ��,twitter,neutral,positive
803,RT Vuzix Smart Glasses Selected to Support NSION's Seamless Situational Awareness Solution for DOD and First Responders $vuzi,twitter,neutral,positive
868,"RT AR Smart Glasses: XR Today Expert Round Table CEO Paul Travers joined industry peers from , and to share insights on the future of . Check it out here: $vuzi",twitter,neutral,positive
661,RT Vuzix feature via : Vuzix Ultralite last up to two days on a single charge. More insight by here: $vuzi,twitter,neutral,positive
472,Telecom Customer Demand for Vuzix Smart Glasses Running on 4G and 5G Networks Continues to Steadily Expand $VUZI,twitter,neutral,positive
828,RT Vuzix smart glasses with ‘Surgeon’s Eye View’ from Brings into the . Read more here: $vuzi,twitter,neutral,positive
829,RT Drive efficiency with Vuzix hands-free vision picking solutions. Learn more about Vuzix M400 smart glasses here: $vuzi,twitter,neutral,positive
676,RT Hongke distributes Vuzix Smart Glasses in the Chinese Market at Mobile World Congress 2023 $vuzi,twitter,neutral,positive
425,"RT Who's going to ? Team Vuzix will be at Booth # 16991! Stop by for a chance to demo Vuzix M400, Blade 2, Shield, and more. Stay tuned for more details to come. $vuzi",twitter,neutral,positive
76,"The stock price of Vuzix (VUZI) increased by 11.38% in the most recent trading session. Why: Vuzix announced that its Vuzix smart glasses have begun to be deployed by Ox, a Vuzix value-added reseller, at one of the world’s largest logistics platform...",news,neutral,positive


In [8]:
google_glasses = preds.loc[((preds['query']=='glasses') & (preds['tag']=='google'))].reset_index(drop=True)
vuzix_glasses = preds.loc[((preds['query']=='glasses') & (preds['tag']=='vuzix'))].reset_index(drop=True)

In [9]:
len(google_glasses),len(vuzix_glasses)

(9850, 2268)

In [40]:
# Vuzix Breakdown
# Look at android nlp disagreement
vuzix_glasses['llm_label'] = vuzix_glasses['llm_label'].apply(lambda x: sentiment_dict[x])
vuzix_glasses['disagreement'] = vuzix_glasses.apply(lambda x: x.android_sentiment!=x.llm_label,axis=1)

In [41]:
vuzix_glasses_positive = vuzix_glasses.loc[vuzix_glasses['llm_label']=='positive'].reset_index(drop=True)

In [43]:
vuzix_glasses['llm_label'].value_counts()/len(vuzix_glasses)

llm_label
positive    0.588624
neutral     0.405203
negative    0.006173
Name: count, dtype: float64

# Google Breakdown 

android_sentiment
neutral     0.475939
negative    0.376041
positive    0.146701

Name: count, dtype: float64



llm_label
neutral     0.476142
negative    0.413503
positive    0.110355
Name: count, dtype: float64

In [ ]:
# Look at android nlp disagreement
google_glasses['llm_label'] = google_glasses['llm_label'].apply(lambda x: sentiment_dict[x])
google_glasses['disagreement'] = google_glasses.apply(lambda x: x.android_sentiment!=x.llm_label,axis=1)

In [29]:
google_glasses_negative = google_glasses.loc[google_glasses['llm_label']=='negative'].reset_index(drop=True)

In [20]:
google_glasses_negative['updated_source'].value_counts()/len(google_glasses_negative)

updated_source
twitter      0.583354
reddit       0.231034
news         0.184139
instagram    0.001473
Name: count, dtype: float64

In [37]:
google_glasses['llm_label'].value_counts()

llm_label
neutral     4690
negative    4073
positive    1087
Name: count, dtype: int64

In [36]:
google_glasses_negative.loc[google_glasses_negative['disagreement']==True].sample(n=10,random_state=42)[['text','updated_source','android_sentiment','llm_label']]

,text,updated_source,android_sentiment,llm_label
1188,Looks better than the Google glass!,reddit,positive,negative
1613,"Marketing is important but at the end of the day a product needs to be desired to sell. Just look at Google Glass, Segway.",reddit,neutral,negative
3349,"""Can Microsoft succeed where Google failed? The company has recently filed a patent for augmented reality (AR) glasses and looks like it's going to try bettering Google Glass. """,twitter,neutral,negative
3888,"Google Glasses has more of a nerdy weird vibe to it, Apple’s is rather a kind of futuristic weird vibe, also looks much more “refined” imo.",twitter,neutral,negative
616,"...launch its own AR glasses (codenamed Project Iris), but it laid off a lot of people in the AR/VR division and then decided to make software called Micro XR for AR headsets from other brands (like Samsung). It also discontinued the enterprise version of Google...",news,neutral,negative
245,"...Insider, The Verge) The post Google Sends Project Iris AR Glasses To The Graveyard. appeared first on .",news,neutral,negative
3976,Find out what's up with the Superhexa Vision �� - a smart glass that wants to out smart glass Google Glass and Snap Spectacles. This is my early hands-on first impressions.,twitter,neutral,negative
1705,RT Why is Jack there? There's also the guy next to him with the Google glass. That's still a thing?,twitter,neutral,negative
604,"Samsung's upcoming AR/XR headset could have a shaky start - SamMobile...launch its own AR glasses (codenamed Project Iris), but it laid off a lot of people in the AR/VR division and then decided to make software called Micro XR for AR headsets from other brands (like Samsung). It also discontinued t",news,neutral,negative
145,"Google Reportedly Cancels ‘Project Iris' Augmented Reality Smart GlassesNeSan Francisco, NFAPost: Google is pulling the plug on its project to build a pair of augmented reality (AR) glasses, according to a report by Business Insider. Google shelved its plans for the glasses, internally known as Iris",news,neutral,negative
